## Restaurant Recommendation System

This Restaurant Recommendation System is used to analyze the available Restaurant data in a perticular city and adivice the user 
with the place details where he can start the new Restaurant. 

#### Importing necessary Libraries

In [2]:
import requests
import pandas as pd
import numpy as np
import random

from geopy.geocoders import Nominatim

from IPython.display import Image
from IPython.core.display import HTML

from pandas.io.json import json_normalize

!conda install -c conda-forge folium=0.5.0 --yes
import folium 

print('Folium installed')
print('Libraries imported')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    openssl-1.1.1g             |       h516909a_0         2.1 MB  conda-forge
    branca-0.4.1               |             py_0          26 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    altair-4.1.0               |             py_1         614 KB  conda-forge
    ca-certificates-2020.6.20  |       hecda079_0         145 KB  conda-forge
    certifi-2020.6.20          |   py36h9f0ad1d_0         151 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    ------------------------------------------------------------
                       

## Define Foursquare credentials 

In [3]:
CLIENT_ID = 'KK2CAPVUQYDEWVDF0LXWA2H5D25OPJTZBIEY1RTQ1MZHGFF4' # your Foursquare ID
CLIENT_SECRET = 'JRVYNHJ3BZSDGQ2DMT04Z3PEQCRKE5EIV0WYQRY1OXGZUVBK' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: KK2CAPVUQYDEWVDF0LXWA2H5D25OPJTZBIEY1RTQ1MZHGFF4
CLIENT_SECRET:JRVYNHJ3BZSDGQ2DMT04Z3PEQCRKE5EIV0WYQRY1OXGZUVBK


Lets plan that we are planning to open a Chinese restaurant. For that first we find what are the chinese restaurants available in Dallas.

In [7]:
address = '3011 Gulden Ln Ste 110,Dallas,TX'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

32.7792282 -96.8278896


## Search for Chinese restaurants

In [8]:
search_query = 'Chinese'
radius = 500
print(search_query + '... Ok!')

Chinese... Ok!


Define the corresponding URL

In [9]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=KK2CAPVUQYDEWVDF0LXWA2H5D25OPJTZBIEY1RTQ1MZHGFF4&client_secret=JRVYNHJ3BZSDGQ2DMT04Z3PEQCRKE5EIV0WYQRY1OXGZUVBK&ll=32.7792282,-96.8278896&v=20180604&query=Chinese&radius=500&limit=30'

In [10]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5f13d9a4e5f51910d6ff2c16'},
 'response': {'venues': [{'id': '5ee910fed7054d000853856f',
    'name': 'Sum Dang Good Chinese',
    'location': {'address': '3011 Gulden Ln',
     'lat': 32.779453,
     'lng': -96.828723,
     'labeledLatLngs': [{'label': 'display',
       'lat': 32.779453,
       'lng': -96.828723}],
     'distance': 81,
     'postalCode': '75212',
     'cc': 'US',
     'city': 'Dallas',
     'state': 'TX',
     'country': 'United States',
     'formattedAddress': ['3011 Gulden Ln',
      'Dallas, TX 75212',
      'United States']},
    'categories': [{'id': '4bf58dd8d48988d145941735',
      'name': 'Chinese Restaurant',
      'pluralName': 'Chinese Restaurants',
      'shortName': 'Chinese',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/asian_',
       'suffix': '.png'},
      'primary': True}],
    'delivery': {'id': '2048307',
     'url': 'https://www.grubhub.com/restaurant/sum-dang-good-3011-gulden-ln-dallas

In [11]:
venues = results['response']['venues']

dataframe = json_normalize(venues)
dataframe.head()

,categories,delivery.id,delivery.provider.icon.name,delivery.provider.icon.prefix,delivery.provider.icon.sizes,delivery.provider.name,delivery.url,hasPerk,id,location.address,...,location.country,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.postalCode,location.state,name,referralId
0,"[{'id': '4bf58dd8d48988d145941735', 'name': 'C...",2048307,/delivery_provider_grubhub_20180129.png,https://fastly.4sqi.net/img/general/cap/,"[40, 50]",grubhub,https://www.grubhub.com/restaurant/sum-dang-go...,False,5ee910fed7054d000853856f,3011 Gulden Ln,...,United States,81,"[3011 Gulden Ln, Dallas, TX 75212, United States]","[{'label': 'display', 'lat': 32.779453, 'lng':...",32.779453,-96.828723,75212,TX,Sum Dang Good Chinese,v-1595136588


In [12]:
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,cc,city,country,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,Sum Dang Good Chinese,Chinese Restaurant,3011 Gulden Ln,US,Dallas,United States,81,"[3011 Gulden Ln, Dallas, TX 75212, United States]","[{'label': 'display', 'lat': 32.779453, 'lng':...",32.779453,-96.828723,75212,TX,5ee910fed7054d000853856f


In [13]:
dataframe_filtered.name

0    Sum Dang Good Chinese
Name: name, dtype: object

In [14]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map centred around the Conrad Hotel

# add a red circle marker to represent the Conrad Hotel
folium.features.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='Conrad Hotel',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

# add the Italian restaurants as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map

## Explore the given Venue

In [15]:
venue_id = '5ee910fed7054d000853856f'
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)
url

'https://api.foursquare.com/v2/venues/5ee910fed7054d000853856f?client_id=KK2CAPVUQYDEWVDF0LXWA2H5D25OPJTZBIEY1RTQ1MZHGFF4&client_secret=JRVYNHJ3BZSDGQ2DMT04Z3PEQCRKE5EIV0WYQRY1OXGZUVBK&v=20180604'

In [16]:
result = requests.get(url).json()
print(result['response']['venue'].keys())
result['response']['venue']

dict_keys(['id', 'name', 'contact', 'location', 'canonicalUrl', 'categories', 'verified', 'stats', 'price', 'likes', 'dislike', 'ok', 'delivery', 'allowMenuUrlEdit', 'beenHere', 'specials', 'photos', 'reasons', 'hereNow', 'createdAt', 'tips', 'shortUrl', 'timeZone', 'listed', 'seasonalHours', 'pageUpdates', 'inbox', 'parent', 'hierarchy', 'attributes'])


{'id': '5ee910fed7054d000853856f',
 'name': 'Sum Dang Good Chinese',
 'contact': {},
 'location': {'address': '3011 Gulden Ln',
  'lat': 32.779453,
  'lng': -96.828723,
  'labeledLatLngs': [{'label': 'display',
    'lat': 32.779453,
    'lng': -96.828723}],
  'postalCode': '75212',
  'cc': 'US',
  'city': 'Dallas',
  'state': 'TX',
  'country': 'United States',
  'formattedAddress': ['3011 Gulden Ln', 'Dallas, TX 75212', 'United States']},
 'canonicalUrl': 'https://foursquare.com/v/sum-dang-good-chinese/5ee910fed7054d000853856f',
 'categories': [{'id': '4bf58dd8d48988d145941735',
   'name': 'Chinese Restaurant',
   'pluralName': 'Chinese Restaurants',
   'shortName': 'Chinese',
   'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/asian_',
    'suffix': '.png'},
   'primary': True}],
 'verified': False,
 'stats': {'tipCount': 0},
 'price': {'tier': 1, 'message': 'Cheap', 'currency': '$'},
 'likes': {'count': 1,
  'groups': [{'type': 'others', 'count': 1, 'items': []}],
  '

In [18]:
result['response']['venue']['tips']['count']

0

In [19]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

KeyError: 'groups'